In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
SEED = 42



In [18]:
original_df = pd.read_csv("original_labels.csv")
eff_kmeans_df = pd.read_csv("EfficientNetB0_KMeans.csv")

# If you have Xception-Birch file:
xcep_birch_df = pd.read_csv("Xception_Birch.csv")

In [26]:
def prepare_data(df, label_column):

    df = df.copy()

    le = LabelEncoder()
    df["label_encoded"] = le.fit_transform(df[label_column])

    num_classes = df["label_encoded"].nunique()

    X = df["image_name"].values
    y = to_categorical(df["label_encoded"], num_classes)

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y,
        test_size=0.3,
        random_state=SEED,
        stratify=df["label_encoded"]
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=0.5,
        random_state=SEED
    )

    return X_train, X_val, X_test, y_train, y_val, y_test, num_classes


In [20]:
import os

MIXED_FOLDER = "CPN_working\mixed_images"   

def load_image(image_name):
    full_path = os.path.join(MIXED_FOLDER, image_name)

    img = tf.keras.preprocessing.image.load_img(
        full_path,
        target_size=(IMG_SIZE, IMG_SIZE)
    )
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    return img


def create_dataset(X, y):
    images = np.array([load_image(name) for name in X])
    return images, y


In [21]:
def build_efficientnet(num_classes):

    base = EfficientNetB0(
        weights=None,            # NO pretrained
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    x = GlobalAveragePooling2D()(base.output)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [22]:
def build_xception(num_classes):

    base = Xception(
        weights=None,           # NO pretrained
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    x = GlobalAveragePooling2D()(base.output)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [23]:
X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(original_df)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)

model_eff_original = build_efficientnet(num_classes)

model_eff_original.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

loss1, acc1 = model_eff_original.evaluate(X_test_img, y_test)
print("EfficientNet trained on ORIGINAL labels Accuracy:", acc1)

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.7641 - loss: 0.6330 - val_accuracy: 0.3355 - val_loss: 3.4596
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.9016 - loss: 0.2739 - val_accuracy: 0.3355 - val_loss: 5.0231
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.9385 - loss: 0.1789 - val_accuracy: 0.3355 - val_loss: 5.6955
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.9519 - loss: 0.1495 - val_accuracy: 0.3355 - val_loss: 3.4960
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.9779 - loss: 0.0684 - val_accuracy: 0.5510 - val_loss: 4.2294
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.9746 - loss: 0.0750 - val_accuracy: 0.7793 - val_loss: 0.9717
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.9735 - loss: 0.0705 - val_accuracy: 0.7143 - val_loss: 2.2695
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.9787 - loss: 0.0687 - val_accu

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(
    eff_kmeans_df,
    label_column="cluster_label"   # <-- adjust if needed
)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)




model_eff_cluster = build_efficientnet(num_classes)

model_eff_cluster.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

loss2, acc2 = model_eff_cluster.evaluate(X_test_img, y_test)

print("\nEfficientNet trained on KMeans labels Accuracy:", acc2)


Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.7948 - loss: 0.5500 - val_accuracy: 0.2883 - val_loss: 4.1297
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.8819 - loss: 0.3198 - val_accuracy: 0.2883 - val_loss: 7.7864
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.9016 - loss: 0.2672 - val_accuracy: 0.2883 - val_loss: 5.7395
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - accuracy: 0.9065 - loss: 0.2496 - val_accuracy: 0.2883 - val_loss: 6.3877
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - accuracy: 0.9248 - loss: 0.1991 - val_accuracy: 0.2883 - val_loss: 5.0916
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9333 - loss: 0.1796 - val_accuracy: 0.3163 - val_loss: 3.4720
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9437 - loss: 0.1539 - val_accuracy: 0.8610 - val_loss: 0.4170
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9549 - loss: 0.1265 - val_accu

In [31]:
# Prepare data (original labels)

X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(
    original_df,
    label_column="label"   # original label column
)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)

# Build model
model_x_original = build_xception(num_classes)

# Train
model_x_original.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# Evaluate
loss3, acc3 = model_x_original.evaluate(X_test_img, y_test)

print("\nXception trained on ORIGINAL labels Accuracy:", acc3)


Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 314s 3s/step - accuracy: 0.8792 - loss: 0.3378 - val_accuracy: 0.3227 - val_loss: 1.1084
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - accuracy: 0.9374 - loss: 0.1786 - val_accuracy: 0.3227 - val_loss: 1.2263
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - accuracy: 0.9549 - loss: 0.1328 - val_accuracy: 0.3227 - val_loss: 1.5976
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 296s 3s/step - accuracy: 0.9634 - loss: 0.1093 - val_accuracy: 0.4668 - val_loss: 1.9971
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - accuracy: 0.9724 - loss: 0.0833 - val_accuracy: 0.6301 - val_loss: 1.3915
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - accuracy: 0.9656 - loss: 0.0952 - val_accuracy: 0.5727 - val_loss: 1.8586
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - accuracy: 0.9770 - loss: 0.0673 - val_accuracy: 0.8929 - val_loss: 0.3868
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - accuracy: 0.9850 - loss: 0.0420 - val_accu

In [33]:
# Prepare data (Birch cluster labels)

X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(
    xcep_birch_df,
    label_column="cluster_label"   # <-- change if different
)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)

# Build model
model_x_cluster = build_xception(num_classes)

# Train
model_x_cluster.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# Evaluate
loss4, acc4 = model_x_cluster.evaluate(X_test_img, y_test)

print("\nXception trained on Birch labels Accuracy:", acc4)


Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 321s 3s/step - accuracy: 0.9552 - loss: 0.1453 - val_accuracy: 0.9439 - val_loss: 0.6226
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 301s 3s/step - accuracy: 0.9724 - loss: 0.0883 - val_accuracy: 0.9439 - val_loss: 0.2381
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 302s 3s/step - accuracy: 0.9825 - loss: 0.0513 - val_accuracy: 0.9439 - val_loss: 0.2633
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 301s 3s/step - accuracy: 0.9820 - loss: 0.0536 - val_accuracy: 0.9439 - val_loss: 0.4310
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 299s 3s/step - accuracy: 0.9803 - loss: 0.0591 - val_accuracy: 0.9541 - val_loss: 0.2266
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 297s 3s/step - accuracy: 0.9855 - loss: 0.0456 - val_accuracy: 0.9758 - val_loss: 0.0556
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 294s 3s/step - accuracy: 0.9869 - loss: 0.0345 - val_accuracy: 0.9719 - val_loss: 0.0794
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 294s 3s/step - accuracy: 0.9872 - loss: 0.0371 - val_accu

In [34]:

# =========================
# FINAL COMPARISON
# =========================

print("\n========== FINAL RESULTS ==========")
print("EfficientNet Original Accuracy:", acc1)
print("EfficientNet Cluster Accuracy :", acc2)
print("Xception Original Accuracy    :", acc3)
print("Xception Cluster Accuracy     :", acc4)


========== FINAL RESULTS ==========
EfficientNet Original Accuracy: 0.9452229142189026
EfficientNet Cluster Accuracy : 0.862420380115509
Xception Original Accuracy    : 0.9579617977142334
Xception Cluster Accuracy     : 0.9745222926139832
